### Switch Models

This notebook provides basic functionality to switch models already trained. An assumption made is that
the model references in database and actual model files are in sync

- PS: Switching to DL model is buggy esp. after training was done recentlty as GPU memory gets hogged (atleast on my machine)

In [1]:
## run this if running locally not requires if you used the docker script
## !pip install python-dotenv
# from dotenv import load_dotenv
# load_dotenv(dotenv_path = '../.env')

In [2]:
import pandas as pd
#from mlcore.dbhelper import read_data
from mlcore.modelops import get_deployment_hist, get_deployed_model
from mlcore.dbhelper import store_data, overwrite_data
from mlcore.modelops import load_model

### See trained and deployed models

In [3]:
# Deployed model
deployed_model_name, deployed_model_obj = get_deployed_model()
print('Deployed Model is :::--> {}'.format(deployed_model_name))

2022-04-30 05:47:02,708:Loaded model stored at ../models/KNN_model_04_30_2022_05_42_59.mdl
2022-04-30 05:47:02,708:Loaded model stored at ../models/KNN_model_04_30_2022_05_42_59.mdl


Deployed Model is :::--> KNN_model_04_30_2022_05_42_59


In [4]:
# Trained Models 
prev_trained_models = get_deployment_hist()
prev_trained_models

,index,final_model_name
0,0,KNN_model_11_21_2021_08_21_23
1,0,N2V_model_11_21_2021_08_21_23
2,0,SEQDL_LSTM_model_11_21_2021_08_21_23
3,0,KNN_model_11_21_2021_08_21_23
4,0,KNN_model_11_21_2021_08_29_05
5,0,KNN_model_11_21_2021_08_21_23
6,0,SEQDL_LSTM_model_11_21_2021_08_21_23
7,0,N2V_model_11_21_2021_08_21_23
8,0,KNN_model_04_09_2022_07_23_57
9,0,KNN_model_04_30_2022_05_42_59


### See trained and deployed models

In [5]:
# Helper Functions which help deploying a model
mldbpath = '../data/mldb.sqlite'

def update_model(deploy_df):

    schema_dict = {
        'deployed_model':deploy_df,
        'hist_deployed_models':deploy_df,
        #'train_report':comparison_result
    }
    for dkey in schema_dict:
        data_to_be_stored = schema_dict[dkey]
        if dkey=='deployed_model':
             overwrite_data(data_to_be_stored, mldbpath, dkey)
        else:
            store_data(data_to_be_stored, mldbpath, dkey)

def switch_model(model_name):
    deployed_model_name, deployed_model_obj = get_deployed_model()
    print('Deployed Model currently set to  :::--> {}'.format(deployed_model_name))
    if model_name==deployed_model_name:
        print('Model :::--> {} is already deployed'.format(deployed_model_name))
    
    else:
        new_model = load_model(model_name)
        
        deploy_df = pd.DataFrame([[model_name]], 
                                 columns =['final_model_name'])
        update_model(deploy_df)
        deployed_model_name, deployed_model_obj = get_deployed_model()
        print('Deployed Model updated to  :::--> {}'.format(model_name))
        


In [6]:
# Generates a log of models you can switch to
prev_trained_models_dict = prev_trained_models['final_model_name'].to_dict()

print('Your Can Change to below model names')
seen = set([deployed_model_name])
for m in prev_trained_models_dict:
    model_name = prev_trained_models_dict[m]
    if model_name not in seen:  
        print('Model Name:::--> {}'.format(model_name))
        seen.add(model_name)


Your Can Change to below model names
Model Name:::--> KNN_model_11_21_2021_08_21_23
Model Name:::--> N2V_model_11_21_2021_08_21_23
Model Name:::--> SEQDL_LSTM_model_11_21_2021_08_21_23
Model Name:::--> KNN_model_11_21_2021_08_29_05
Model Name:::--> KNN_model_04_09_2022_07_23_57


In [8]:
# Uncomment Below and Switch based on above info
model_to_switch_to = 'N2V_model_11_21_2021_08_21_23'
switch_model(model_to_switch_to)

2022-04-30 05:48:31,000:Loaded model stored at ../models/SEQDL_LSTM_model_11_21_2021_08_21_23.mdl
2022-04-30 05:48:31,000:Loaded model stored at ../models/SEQDL_LSTM_model_11_21_2021_08_21_23.mdl
2022-04-30 05:48:34,039:Could not load deployed model it may not exist exception '_UserObject' object has no attribute 'add_slot'
2022-04-30 05:48:34,039:Could not load deployed model it may not exist exception '_UserObject' object has no attribute 'add_slot'
2022-04-30 05:48:34,139:Loaded model stored at ../models/N2V_model_11_21_2021_08_21_23.mdl
2022-04-30 05:48:34,247:Loaded model stored at ../models/N2V_model_11_21_2021_08_21_23.mdl
2022-04-30 05:48:34,247:Loaded model stored at ../models/N2V_model_11_21_2021_08_21_23.mdl
2022-04-30 05:48:34,247:Loaded model stored at ../models/N2V_model_11_21_2021_08_21_23.mdl


Deployed Model currently set to  :::--> None
Deployed Model updated to  :::--> N2V_model_11_21_2021_08_21_23


### PS: 
 - Swithching to Sequence Model Right After training does now work some time because of OOM error
 - Does not work properly for tensorflow models 